#### 01-Analyze 

Statistics the accuracy of building graph algorithm

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import glob, os, time
import json
import numpy as np
from collections import defaultdict

from metadec.dataset.genome import SimGenomeDataset
from metadec.model.dec import DEC
from metadec.utils.utils import load_genomics
from metadec.debug.visualize import store_results
from metadec.utils.metrics import genome_acc

#### Hyperparameter

##### Training

In [ ]:
RESULT_DIR = '../results'
GEN_DATA_DIR = '../data'

# Dir contains raw fasta data
DATASET_DIR = GEN_DATA_DIR
# Specifc dataset or all of them
DATASET_NAME = 'S7'

##### Graph 

In [ ]:
# Hyperparameters
# Follows metaprob
KMERS = [4]
LMER = 30
NUM_SHARED_READS = (5, 45)
ONLY_SEED = True
MAXIMUM_SEED_SIZE = 9000

#### Initialize dirs

In [ ]:
LOG_DIR = f'{RESULT_DIR}\log'
META_INFO =  '../config/dataset_metadata.json'
LOG_DIR = os.path.join(LOG_DIR, 'analysis')

for d in [LOG_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

#### Loading metadata

In [ ]:
raw_dir = os.path.join(DATASET_DIR, 'raw')
processed_dir = os.path.join(DATASET_DIR, 'processed')
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
if DATASET_NAME == 'all':
    raw_datasets = glob.glob(raw_dir + '/*.fna')
else:
    raw_datasets = [os.path.join(raw_dir, DATASET_NAME + '.fna')]

# Mapping of dataset and its corresponding number of clusters
with open(META_INFO, 'r') as f:
    n_clusters_mapping = json.load(f)['simulated']

In [ ]:
raw_datasets

#### Running

In [ ]:
dataset = '../data/raw/S1.fna'

In [ ]:
def analyze_group_label(groups, labels):
    # Assign groups labels from reads labels
    # Dominant number of each label is the label of group
    # [0 0 0 0 1 1 0] -> group has label 0 (5 > 2)
    grps_label = []
    groups_label_stats = []
    for group in groups:
        lb_type = defaultdict(lambda: 0)
        for node in group:
            lb_type[labels[node]] += 1
        max_val = 0
        key = -1
        for kv in lb_type.items():
            if kv[1] > max_val:
                max_val = kv[1]
                key = kv[0]
        if key != -1:
            groups_label_stats.append(max_val/sum([kv[1] for kv in lb_type.items()]))
            
    return sum(groups_label_stats)/len(groups_label_stats)

In [ ]:
# Get some parameters
dataset_name = os.path.basename(dataset).split('.fna')[0]

num_shared_read = NUM_SHARED_READS[1] if 'R' in dataset_name else NUM_SHARED_READS[0]
is_deserialize = os.path.exists(os.path.join(processed_dir, dataset_name + '.json'))
n_clusters = n_clusters_mapping[dataset_name]

seed_kmer_features, labels, groups, seeds = load_genomics(
    dataset,
    kmers=KMERS,
    lmer=LMER,
    maximum_seed_size=MAXIMUM_SEED_SIZE,
    num_shared_reads=num_shared_read,
    is_deserialize=is_deserialize,
    is_serialize=~is_deserialize,
    is_normalize=True,
    only_seed=ONLY_SEED,
    graph_file=os.path.join(processed_dir, dataset_name + '.json')
)

In [ ]:
analyze_group_label(groups, labels)